Copyright 2021-2023 @ Shenzhen Bay Laboratory & Peking University & Huawei Technologies Co., Ltd

This code is a part of MindSPONGE:
MindSpore Simulation Package tOwards Next Generation molecular modelling.

MindSPONGE is open-source software based on the AI-framework:
MindSpore (https://www.mindspore.cn/)

Licensed under the Apache License, Version 2.0 (the "License");
you may not use this file except in compliance with the License.

You may obtain a copy of the License at http://www.apache.org/licenses/LICENSE-2.0

Unless required by applicable law or agreed to in writing, software
distributed under the License is distributed on an "AS IS" BASIS,
WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.

See the License for the specific language governing permissions and
limitations under the License.

MindSPONGE advanced tutorial 03: Energy wrapper and integrated tempering sampling (ITS).

In [1]:
from mindspore import context
from mindsponge import Sponge
from mindsponge import ForceField
from mindsponge import set_global_units
from mindsponge import Protein
from mindsponge import WithEnergyCell
from mindsponge import UpdaterMD
from mindsponge.optimizer import SteepestDescent
from mindsponge.control import VelocityVerlet
from mindsponge.callback import WriteH5MD, RunInfo
from mindsponge.control import Langevin
from mindsponge.sampling import ITS
from mindsponge.function import VelocityGenerator
from mindsponge.colvar import Torsion

[WARNING] ME(189726:140331995714816,MainProcess):2023-03-24-17:52:25.361.797 [mindspore/run_check/_check_version.py:75] Can not found cuda libs, please confirm that the correct cuda version has been installed, you can refer to the installation guidelines: https://www.mindspore.cn/install


In [2]:
context.set_context(mode=context.PYNATIVE_MODE, device_target="GPU")
set_global_units('nm', 'kj/mol')

In [3]:
system = Protein(pdb='alad.pdb')
potential = ForceField(system, 'AMBER.FF14SB')

In [4]:
phi = Torsion([4, 6, 8, 14])
psi = Torsion([6, 8, 14, 16])

In [5]:
min_opt = SteepestDescent(system.trainable_params(), 1e-7)
mini = Sponge(system, potential, min_opt, metrics={'phi': phi, 'psi': psi})

[MindSPONGE] Using fully connected neighbour list (not updated).


In [6]:
run_info = RunInfo(10)
mini.run(100, callbacks=[run_info])

[MindSPONGE] Started simulation at 2023-03-24 17:52:34
[MindSPONGE] Step: 0, E_pot: -55.75643, phi: -3.140886, psi: -3.1410766
[MindSPONGE] Step: 10, E_pot: -58.41481, phi: -3.1339903, psi: -3.1368215
[MindSPONGE] Step: 20, E_pot: -60.197777, phi: -3.1276336, psi: -3.1337032
[MindSPONGE] Step: 30, E_pot: -61.60443, phi: -3.1219037, psi: -3.1313035
[MindSPONGE] Step: 40, E_pot: -62.77914, phi: -3.1167603, psi: -3.1294105
[MindSPONGE] Step: 50, E_pot: -63.786407, phi: -3.1121268, psi: -3.1279025
[MindSPONGE] Step: 60, E_pot: -64.66443, phi: -3.107924, psi: -3.126701
[MindSPONGE] Step: 70, E_pot: -65.43921, phi: -3.1040812, psi: -3.125749
[MindSPONGE] Step: 80, E_pot: -66.13011, phi: -3.100538, psi: -3.125002
[MindSPONGE] Step: 90, E_pot: -66.75129, phi: -3.0972445, psi: -3.1244242
[MindSPONGE] Finished simulation at 2023-03-24 17:52:38
[MindSPONGE] Simulation time: 4.40 seconds.
--------------------------------------------------------------------------------


In [7]:
its = ITS(
    sim_temp=300,
    temp_min=270,
    temp_max=470,
    temp_bin=100,
    update_pace=100,
    unlinear_temp=True,
)

In [8]:
sim = WithEnergyCell(system, potential, wrapper=its)

[MindSPONGE] Using fully connected neighbour list (not updated).


In [9]:
vgen = VelocityGenerator(300)
velocity = vgen(system.coordinate.shape, system.atom_mass)
opt = UpdaterMD(
    system,
    integrator=VelocityVerlet(system),
    thermostat=Langevin(system, 300),
    time_step=1e-3,
    velocity=velocity
)


In [10]:
md = Sponge(sim, optimizer=opt, metrics={'phi': phi, 'psi': psi})

In [11]:
cb_h5md = WriteH5MD(system, 'tutorial_a03.h5md', save_freq=10)
md.run(1000, callbacks=[run_info, cb_h5md])

[MindSPONGE] Started simulation at 2023-03-24 17:52:45
[MindSPONGE] Step: 0, E_pot: -79.554825, E_kin: 112.45297, E_tot: 32.898148, Temperature: 429.36456, phi: -3.109254, psi: -3.1288488
[MindSPONGE] Step: 10, E_pot: -89.68758, E_kin: 35.787235, E_tot: -53.90034, Temperature: 136.64175, phi: 3.0601716, psi: -3.141152
[MindSPONGE] Step: 20, E_pot: -128.85558, E_kin: 75.28584, E_tot: -53.569733, Temperature: 287.45416, phi: 3.1299484, psi: -3.0893805
[MindSPONGE] Step: 30, E_pot: -109.16362, E_kin: 58.659058, E_tot: -50.504562, Temperature: 223.97026, phi: -3.009509, psi: -3.0363805
[MindSPONGE] Step: 40, E_pot: -121.480675, E_kin: 69.40384, E_tot: -52.076836, Temperature: 264.99567, phi: -2.935812, psi: -3.0161312
[MindSPONGE] Step: 50, E_pot: -90.83605, E_kin: 52.4665, E_tot: -38.369553, Temperature: 200.32602, phi: -2.9698148, psi: -3.1073575
[MindSPONGE] Step: 60, E_pot: -123.61399, E_kin: 78.51181, E_tot: -45.10218, Temperature: 299.77142, phi: -3.0945632, psi: 3.0413573
[MindSPONG